In [ ]:
# Data Handling and Processing
import pandas as pd
import numpy as np
import os
from pathlib import Path
import glob
import shutil

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

# Image Processing
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img, save_img

# Data Transformation and Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# Model Training and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

# Model Building
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, MaxPooling2D, \
                                    Permute, TimeDistributed, Bidirectional, GRU, SimpleRNN, LSTM, \
                                    GlobalAveragePooling2D, Input, concatenate, multiply
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K


In [ ]:
# Path to the Excel file
excel_path = "C:\\Users\\dktjw\\OneDrive\\바탕 화면\\Brain hemorrhage detection model\\hemorrhage_diagnosis_final.xlsx"

# Path to the image folder
image_base_path = "C:\\Users\\dktjw\\OneDrive\\바탕 화면\\Brain hemorrhage detection model\\CT_bone_data"

# Read the Excel file
df = pd.read_excel(excel_path)

# Set label column names
labels = {
    'Intraventricular(뇌실내출혈)': 'Intraventricular',
    'Intraparenchymal(뇌실질내출혈)': 'Intraparenchymal',
    'Subarachnoid(지주막하출혈)': 'Subarachnoid',
    'Epidural(경막외출혈)': 'Epidural',
    'Subdural(경막하출혈)': 'Subdural',
    'No_Hemorrhage': 'No hemorrhage'
}

# Initialize dictionaries to store results
brain_data = []
bone_data = []

# Function to convert to three-digit number
def make_folder_name(folder_num):
    return str(folder_num).zfill(3)

# Labeling image files
for index, row in df.iterrows():
    folder_name = make_folder_name(row['PatientNumber'])  # Convert folder name to three digits
    image_name = str(row['SliceNumber']) + '.jpg'  # Add extension to image name

    # Determine label
    label = None
    for col, label_name in labels.items():
        if row[col] == 1:
            label = label_name
            break
    
    # Label images in brain and bone folders
    for subfolder in ['brain', 'bone']:
        image_folder_path = os.path.join(image_base_path, folder_name, subfolder)
        image_path = os.path.join(image_folder_path, image_name)

        # Save results
        if subfolder == 'brain':
            brain_data.append((image_path, label))
        else:
            bone_data.append((image_path, label))

# Create DataFrames for Brain_Data and Bone_Data
CT_Data = pd.DataFrame(brain_data, columns=['Image', 'Label'])
Bone_Data = pd.DataFrame(bone_data, columns=['Image', 'Label'])

In [ ]:
# FOR CHECKING NUMBER OF DATA LABEL
print(CT_Data["Label"].value_counts())
print(Bone_Data["Label"].value_counts())

In [ ]:
# Combine Brain and Bone datasets
combined_data = pd.concat([CT_Data, Bone_Data], axis=1)

# Split into Train and Test sets
Train_Data, Test_Data = train_test_split(combined_data, train_size=0.8, shuffle=True, random_state=42)

# Separate Brain data and Bone data in each Train and Test set
Train_Data_Brain = Train_Data.iloc[:, [0, 1]]  # Select first and second columns
Train_Data_Bone = Train_Data.iloc[:, [2, 3]]   # Select third and fourth columns
Test_Data_Brain = Test_Data.iloc[:, [0, 1]]    # Select first and second columns
Test_Data_Bone = Test_Data.iloc[:, [2, 3]]     # Select third and fourth columns

# Count the number of samples for each label in the Test set
brain_test_label_counts = Test_Data_Brain['Label'].value_counts()
bone_test_label_counts = Test_Data_Bone['Label'].value_counts()

# Print results
print("\n\nBRAIN TEST LABEL COUNTS:")
print(brain_test_label_counts)

print("\nBONE TEST LABEL COUNTS:")
print(bone_test_label_counts)

In [ ]:
## DATA AUGMENTATION

# Data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def augment_data(brain_df, bone_df, output_folder, target_count=2500):
    os.makedirs(output_folder, exist_ok=True)

    augmented_brain_data = []
    augmented_bone_data = []

    for label in brain_df['Label'].unique():
        brain_label_df = brain_df[brain_df['Label'] == label]
        bone_label_df = bone_df[bone_df['Label'] == label]

        current_count = len(brain_label_df)
        label_brain_folder = os.path.join(output_folder, f"{label}_brain")
        label_bone_folder = os.path.join(output_folder, f"{label}_bone")
        os.makedirs(label_brain_folder, exist_ok=True)
        os.makedirs(label_bone_folder, exist_ok=True)

        # Sequentially copy existing images to label folders
        for idx, (brain_row, bone_row) in enumerate(zip(brain_label_df.itertuples(), bone_label_df.itertuples()), start=1):
            brain_img_path = brain_row.Image
            bone_img_path = bone_row.Image

            dest_brain_path = os.path.join(label_brain_folder, f"{label}_brain{idx}.jpg")
            dest_bone_path = os.path.join(label_bone_folder, f"{label}_bone{idx}.jpg")

            shutil.copy(brain_img_path, dest_brain_path)
            shutil.copy(bone_img_path, dest_bone_path)

            augmented_brain_data.append((dest_brain_path, label))
            augmented_bone_data.append((dest_bone_path, label))

        # Augment to make up for missing count
        augment_count = target_count - current_count
        if augment_count > 0:
            brain_imgs_to_augment = brain_label_df['Image'].tolist()
            bone_imgs_to_augment = bone_label_df['Image'].tolist()
            augment_per_img = augment_count // len(brain_imgs_to_augment) + 1

            for brain_img_path, bone_img_path in zip(brain_imgs_to_augment, bone_imgs_to_augment):
                brain_img = load_img(brain_img_path)
                bone_img = load_img(bone_img_path)

                x_brain = img_to_array(brain_img)
                x_bone = img_to_array(bone_img)

                x_brain = x_brain.reshape((1,) + x_brain.shape)
                x_bone = x_bone.reshape((1,) + x_bone.shape)

                i = 0
                for brain_batch, bone_batch in zip(datagen.flow(x_brain, batch_size=1), datagen.flow(x_bone, batch_size=1)):
                    if i >= augment_per_img or len(os.listdir(label_brain_folder)) >= target_count:
                        break

                    new_brain_img = array_to_img(brain_batch[0])
                    new_bone_img = array_to_img(bone_batch[0])

                    current_index = len(os.listdir(label_brain_folder)) + 1

                    new_brain_img_name = f"{label}_brain{current_index}.jpg"
                    new_bone_img_name = f"{label}_bone{current_index}.jpg"

                    new_brain_img_path = os.path.join(label_brain_folder, new_brain_img_name)
                    new_bone_img_path = os.path.join(label_bone_folder, new_bone_img_name)

                    save_img(new_brain_img_path, brain_batch[0])
                    save_img(new_bone_img_path, bone_batch[0])

                    augmented_brain_data.append((new_brain_img_path, label))
                    augmented_bone_data.append((new_bone_img_path, label))

                    i += 1

    # Create augmented DataFrames
    augmented_brain_df = pd.DataFrame(augmented_brain_data, columns=['Image', 'Label'])
    augmented_bone_df = pd.DataFrame(augmented_bone_data, columns=['Image', 'Label'])

    return augmented_brain_df, augmented_bone_df

def find_missing_numbers(nums, target_count=2500):
    """
    Find and return missing numbers in the given list.
    """
    missing_nums = [i for i in range(1, target_count + 1) if i not in nums]
    return missing_nums

def fill_missing_images(df, output_folder, target_count=2500):
    """
    Find missing numbers in the given DataFrame and augment data to fill those numbers.
    """
    global datagen

    augmented_brain_data = []
    augmented_bone_data = []

    for label in df['Label'].unique():
        label_folder = os.path.join(output_folder, f"{label}_brain")
        existing_files = [int(f.split(label + "_brain")[1].split(".jpg")[0]) for f in os.listdir(label_folder)]
        missing_nums = find_missing_numbers(existing_files, target_count)

        if not missing_nums:
            # Already have 2500 images
            continue

        for num in missing_nums:
            brain_img_path = df[df['Label'] == label]['Image'].sample().values[0]
            bone_img_path = brain_img_path.replace('brain', 'bone')

            brain_img = load_img(brain_img_path)
            bone_img = load_img(bone_img_path)

            x_brain = img_to_array(brain_img)
            x_bone = img_to_array(bone_img)

            x_brain = x_brain.reshape((1,) + x_brain.shape)
            x_bone = x_bone.reshape((1,) + x_bone.shape)

            brain_batch = next(datagen.flow(x_brain, batch_size=1))
            bone_batch = next(datagen.flow(x_bone, batch_size=1))

            new_brain_img = array_to_img(brain_batch[0])
            new_bone_img = array_to_img(bone_batch[0])

            new_brain_img_name = f"{label}_brain{num}.jpg"
            new_bone_img_name = f"{label}_bone{num}.jpg"

            new_brain_img_path = os.path.join(label_folder, new_brain_img_name)
            new_bone_img_path = os.path.join(label_folder.replace('brain', 'bone'), new_bone_img_name)

            save_img(new_brain_img_path, brain_batch[0])
            save_img(new_bone_img_path, bone_batch[0])

            augmented_brain_data.append((new_brain_img_path, label))
            augmented_bone_data.append((new_bone_img_path, label))

    # Add newly created images to the existing DataFrame
    augmented_brain_df = pd.concat([df, pd.DataFrame(augmented_brain_data, columns=['Image', 'Label'])])
    augmented_bone_df = pd.concat([df, pd.DataFrame(augmented_bone_data, columns=['Image', 'Label'])])

    return augmented_brain_df, augmented_bone_df

# Example DataFrames creation (replace with actual data)
# Train_Data_Brain = pd.DataFrame([...])  # Actual Train Brain DataFrame
# Train_Data_Bone = pd.DataFrame([...])  # Actual Train Bone DataFrame

# Set folder to save augmented data
output_folder = 'Augmented_Data'

# Perform data augmentation
Augmented_Train_Data_Brain, Augmented_Train_Data_Bone = augment_data(Train_Data_Brain, Train_Data_Bone, output_folder)

# Fill missing images if any and augment
filled_Augmented_Train_Data_Brain, filled_Augmented_Train_Data_Bone = fill_missing_images(Augmented_Train_Data_Brain, output_folder)
filled_Augmented_Train_Data_Brain, filled_Augmented_Train_Data_Bone = fill_missing_images(Augmented_Train_Data_Bone, output_folder)

# Print final results
print("Augmented Train Data Brain:")
print(filled_Augmented_Train_Data_Brain.head())

print("\nAugmented Train Data Bone:")
print(filled_Augmented_Train_Data_Bone.head())


## Code to reload the generated image files

def load_augmented_images(output_folder):
    brain_data = []
    bone_data = []

    for label_folder in os.listdir(output_folder):
        if label_folder.endswith('_brain'):
            label = label_folder.split('_brain')[0]
            brain_folder_path = os.path.join(output_folder, label_folder)
            bone_folder_path = brain_folder_path.replace('_brain', '_bone')
            
            for brain_img_file in os.listdir(brain_folder_path):
                brain_img_path = os.path.join(brain_folder_path, brain_img_file)
                bone_img_path = os.path.join(bone_folder_path, brain_img_file.replace('_brain', '_bone'))
                
                brain_data.append((brain_img_path, label))
                bone_data.append((bone_img_path, label))

    brain_df = pd.DataFrame(brain_data, columns=['Image', 'Label'])
    bone_df = pd.DataFrame(bone_data, columns=['Image', 'Label'])

    return brain_df, bone_df

# Folder where augmented data is saved
output_folder = 'Augmented_Data'

# Load images from the folder and create DataFrame
Augmented_Train_Data_Brain, Augmented_Train_Data_Bone = load_augmented_images(output_folder)

# Print final results
print("Augmented Train Data Brain:")
print(Augmented_Train_Data_Brain.head())

print("\nAugmented Train Data Bone:")
print(Augmented_Train_Data_Bone.head())

# Check the number of images
print("\nTotal number of brain images:", len(Augmented_Train_Data_Brain))
print("Total number of bone images:", len(Augmented_Train_Data_Bone))

In [ ]:
# Function to preprocess the image
def preprocess_image(image_path, target_size):
    try:
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Function to load images from dataframe
def load_images_from_dataframe(df, target_size):
    images = []
    labels = []
    
    for index, row in df.iterrows():
        image_path = row['Image']
        image = preprocess_image(image_path, target_size)
        if image is not None:
            images.append(image)
            labels.append(row['Label'])
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Parameters
IMG_SIZE = (128, 128)

# Load images and labels for brain train dataset
brain_train_images, brain_train_labels = load_images_from_dataframe(Agumented_Train_Data_Brain, IMG_SIZE)

# Load images and labels for brain test dataset
brain_test_images, brain_test_labels = load_images_from_dataframe(Test_Data_Brain, IMG_SIZE)

# Load images and labels for bone train dataset
bone_train_images, bone_train_labels = load_images_from_dataframe(Agumented_Train_Data_Bone, IMG_SIZE)

# Load images and labels for bone test dataset
bone_test_images, bone_test_labels = load_images_from_dataframe(Test_Data_Bone, IMG_SIZE)

# Normalize images
brain_train_images = brain_train_images / 255.0
brain_test_images = brain_test_images / 255.0
bone_train_images = bone_train_images / 255.0
bone_test_images = bone_test_images / 255.0

# Label encoding and one-hot encoding
def encode_labels(labels):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    one_hot_encoded = to_categorical(integer_encoded)
    return one_hot_encoded

brain_train_labels_encoded = encode_labels(brain_train_labels)
brain_test_labels_encoded = encode_labels(brain_test_labels)
bone_train_labels_encoded = encode_labels(bone_train_labels)
bone_test_labels_encoded = encode_labels(bone_test_labels)

In [ ]:
# Build the enhanced CNN model for brain images
input_brain = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_brain)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = Dropout(0.3)(x1)

x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = Dropout(0.3)(x1)

x1 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = Dropout(0.4)(x1)

x1 = Flatten()(x1)

# Build the enhanced CNN model for bone images
input_bone = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x2 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_bone)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D((2, 2))(x2)
x2 = Dropout(0.3)(x2)

x2 = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D((2, 2))(x2)
x2 = Dropout(0.3)(x2)

x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x2)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D((2, 2))(x2)
x2 = Dropout(0.4)(x2)

x2 = Flatten()(x2)

# Classification layer for brain-only model
output_brain_only = Dense(6, activation='softmax')(x1)

# Combine the outputs using concatenate
combined_concat = concatenate([x1, x2])

# Product fusion
combined_product = multiply([x1, x2])

# Classification layer for concatenate fusion
combined_concat = Dense(64, activation='relu')(combined_concat)
combined_concat = Dropout(0.5)(combined_concat)
output_concat = Dense(6, activation='softmax')(combined_concat)

# Classification layer for product fusion
combined_product = Dense(64, activation='relu')(combined_product)
combined_product = Dropout(0.5)(combined_product)
output_product = Dense(6, activation='softmax')(combined_product)


# Define models
model_concat = Model(inputs=[input_brain, input_bone], outputs=output_concat)
model_product = Model(inputs=[input_brain, input_bone], outputs=output_product)
model_brain_only = Model(inputs=input_brain, outputs=output_brain_only)


# Compile models
model_concat.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_product.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_brain_only.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
# model_concat.summary()
# model_product.summary()
# model_brain_only.summar()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint_concat = ModelCheckpoint('best_model_concat.keras', monitor='val_accuracy', save_best_only=True, mode='max')
model_checkpoint_product = ModelCheckpoint('best_model_product.keras', monitor='val_accuracy', save_best_only=True, mode='max')
model_checkpoint_brain_only = ModelCheckpoint('best_model_brain_only.keras', monitor='val_accuracy', save_best_only=True, mode='max')


# Train the models
history_concat = model_concat.fit([brain_train_images, bone_train_images], brain_train_labels_encoded, 
                                  epochs=50, 
                                  batch_size=32, 
                                  validation_data=([brain_test_images, bone_test_images], brain_test_labels_encoded), 
                                  callbacks=[early_stopping, model_checkpoint_concat])

history_product = model_product.fit([brain_train_images, bone_train_images], brain_train_labels_encoded, 
                                    epochs=50, 
                                    batch_size=32, 
                                    validation_data=([brain_test_images, bone_test_images], brain_test_labels_encoded), 
                                    callbacks=[early_stopping, model_checkpoint_product])

history_brain_only = model_brain_only.fit(brain_train_images, brain_train_labels_encoded,
                                          epochs=50,
                                          batch_size=32,
                                          validation_data=(brain_test_images, brain_test_labels_encoded),
                                          callbacks=[early_stopping])


# Plot training & validation accuracy values for all three models
plt.plot(history_brain_only.history['accuracy'])
plt.plot(history_brain_only.history['val_accuracy'])
plt.plot(history_concat.history['accuracy'])
plt.plot(history_concat.history['val_accuracy'])
plt.plot(history_product.history['accuracy'])
plt.plot(history_product.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Brain Only Train', 'Brain Only Val', 'Concat Train', 'Concat Val', 'Product Train', 'Product Val'], loc='upper left')
plt.show()

# Plot training & validation loss values for all three models
plt.plot(history_brain_only.history['loss'])
plt.plot(history_brain_only.history['val_loss'])
plt.plot(history_concat.history['loss'])
plt.plot(history_concat.history['val_loss'])
plt.plot(history_product.history['loss'])
plt.plot(history_product.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Brain Only Train', 'Brain Only Val', 'Concat Train', 'Concat Val', 'Product Train', 'Product Val'], loc='upper left')
plt.show()

# Print the best accuracy of each model
best_val_accuracy_brain_only = max(history_brain_only.history['accuracy'])
best_val_accuracy_concat = max(history_concat.history['accuracy'])
best_val_accuracy_product = max(history_product.history['accuracy'])

print(f'Best Validation Accuracy for Brain Only Model: {best_val_accuracy_brain_only:.4f}')
print(f'Best Validation Accuracy for Concatenate Model: {best_val_accuracy_concat:.4f}')
print(f'Best Validation Accuracy for Product Model: {best_val_accuracy_product:.4f}')

In [ ]:
def labels_to_string(labels):
    return np.argmax(labels, axis=1)

def evaluate_and_compare_model(model_path, brain_test_images, bone_test_images, brain_test_labels_encoded, model_type='combined'):
    best_model = load_model(model_path)
    
    if model_type == 'brain_only':
        test_loss, test_accuracy = best_model.evaluate(brain_test_images, brain_test_labels_encoded)
        predicted_labels = best_model.predict(brain_test_images)
    else:
        test_loss, test_accuracy = best_model.evaluate([brain_test_images, bone_test_images], brain_test_labels_encoded)
        predicted_labels = best_model.predict([brain_test_images, bone_test_images])
    
    print(f"Test Accuracy: {test_accuracy}, Test Loss: {test_loss}")
    
    predicted_labels_binary = (predicted_labels > 0.5).astype(int)
    
    brain_test_labels_decoded = labels_to_string(brain_test_labels_encoded)
    predicted_labels_decoded = labels_to_string(predicted_labels_binary)
    
    test_accuracy = accuracy_score(brain_test_labels_decoded, predicted_labels_decoded)
    
    comparison = pd.DataFrame({
        'Actual': brain_test_labels_decoded,
        'Predicted': predicted_labels_decoded
    })
    
    return comparison

comparison_brain_only = evaluate_and_compare_model('best_model_brain_only.keras', brain_test_images, None, brain_test_labels_encoded, model_type='brain_only')
comparison_concat = evaluate_and_compare_model('best_model_concat.keras', brain_test_images, bone_test_images, brain_test_labels_encoded, model_type='combined')
comparison_product = evaluate_and_compare_model('best_model_product.keras', brain_test_images, bone_test_images, brain_test_labels_encoded, model_type='combined')

# Save the comparisons to CSV and Excel files
comparison_brain_only.to_excel('prediction_comparison_brain_only.xlsx', index=False)
comparison_concat.to_excel('prediction_comparison_concat.xlsx', index=False)
comparison_product.to_excel('prediction_comparison_product.xlsx', index=False)

print("Comparison for Brain Only model:")
print(comparison_brain_only.head())

print("Comparison for Concat model:")
print(comparison_concat.head())

print("Comparison for Product model:")
print(comparison_product.head())


In [ ]:
class_labels = ['Epidural', 'Intraparenchymal', 'Intraventricular', 'No hemorrhage', 'Subarachnoid', 'Subdural']

# Create confusion matrices by comparing predicted and actual values
conf_matrix_brain_only = confusion_matrix(labels_to_string(brain_test_labels_encoded), labels_to_string(model_brain_only.predict(brain_test_images)))
conf_matrix_concat = confusion_matrix(labels_to_string(brain_test_labels_encoded), labels_to_string(model_concat.predict([brain_test_images, bone_test_images])))
conf_matrix_product = confusion_matrix(labels_to_string(brain_test_labels_encoded), labels_to_string(model_product.predict([brain_test_images, bone_test_images])))

# Visualize with heatmap
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.heatmap(conf_matrix_brain_only, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix for Brain Only Model')
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')

plt.subplot(1, 3, 2)
sns.heatmap(conf_matrix_concat, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix for Concat Model')
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')

plt.subplot(1, 3, 3)
sns.heatmap(conf_matrix_product, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix for Product Model')
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')

plt.tight_layout()
plt.show()


XAI(Gradient-CAM)

In [ ]:
# Load the pre-trained models
model_brain_only = load_model('best_model_brain_only.keras')
model_concat = load_model('best_model_concat.keras')
model_product = load_model('best_model_product.keras')

# Function to preprocess the image
def preprocess_image(image_path, target_size):
    try:
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Function to get Grad-CAM heatmap
def get_grad_cam(model, img_array_brain, img_array_bone, category_index, layer_name, model_type='combined'):
    if model_type == 'brain_only':
        grad_model = Model(inputs=model.inputs, outputs=[model.get_layer(layer_name).output, model.output])
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(img_array_brain)
            loss = predictions[:, category_index]
        grads = tape.gradient(loss, conv_outputs)
    else:
        grad_model = Model(inputs=model.inputs, outputs=[model.get_layer(layer_name).output, model.output])
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model([img_array_brain, img_array_bone])
            loss = predictions[:, category_index]
        grads = tape.gradient(loss, conv_outputs)
        
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    pooled_grads = pooled_grads.numpy()
    conv_outputs = conv_outputs.numpy()
    
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]
        
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    if np.max(heatmap) != 0:
        heatmap /= np.max(heatmap)
    heatmap = cv2.resize(heatmap, (img_array_brain.shape[2], img_array_brain.shape[1]))
    heatmap = np.uint8(255 * heatmap)
    return heatmap

# Example visualization for each category
categories = ['no hemorrhage', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
brain_layer_name = 'conv2d_1'  # Layer name for brain images in brain_only model
combined_brain_layer_name = 'conv2d_1'  # Layer name for brain images in combined models
combined_bone_layer_name = 'conv2d_3'   # Layer name for bone images in combined models

# Use one of the paths from your test set
img_path = Test_Data_Brain.iloc[0]['Image']  # Path to a test brain image
bone_img_path = Test_Data_Bone.iloc[0]['Image']  # Path to a test bone image

# Preprocess the images
brain_img = preprocess_image(img_path, IMG_SIZE)
bone_img = preprocess_image(bone_img_path, IMG_SIZE)

if brain_img is not None and bone_img is not None:
    brain_img_array = np.expand_dims(brain_img, axis=0) / 255.0
    bone_img_array = np.expand_dims(bone_img, axis=0) / 255.0

    plt.figure(figsize=(20, 40))
    for i, category in enumerate(categories):
        # Get Grad-CAM heatmap for brain image for each category (Brain Only Model)
        brain_heatmap_brain_only = get_grad_cam(model_brain_only, brain_img_array, None, category_index=i, layer_name=brain_layer_name, model_type='brain_only')
        
        # Get Grad-CAM heatmap for brain image for each category (Concat Model)
        brain_heatmap_concat = get_grad_cam(model_concat, brain_img_array, bone_img_array, category_index=i, layer_name=combined_brain_layer_name)

        # Get Grad-CAM heatmap for bone image for each category (Concat Model)
        bone_heatmap_concat = get_grad_cam(model_concat, brain_img_array, bone_img_array, category_index=i, layer_name=combined_bone_layer_name)
        
        # Get Grad-CAM heatmap for brain image for each category (Product Model)
        brain_heatmap_product = get_grad_cam(model_product, brain_img_array, bone_img_array, category_index=i, layer_name=combined_brain_layer_name)

        # Get Grad-CAM heatmap for bone image for each category (Product Model)
        bone_heatmap_product = get_grad_cam(model_product, brain_img_array, bone_img_array, category_index=i, layer_name=combined_bone_layer_name)
        
        # Display the Grad-CAM heatmap overlayed on the original image (Brain Only Model)
        plt.subplot(len(categories), 4, 4 * i + 1)
        plt.imshow(brain_img / 255.0)
        plt.imshow(brain_heatmap_brain_only, cmap='jet', alpha=0.5)
        plt.title(f'Grad-CAM Brain Only: Brain Image - {category}')

        # Display the Grad-CAM heatmap overlayed on the original image (Concat Model)
        plt.subplot(len(categories), 4, 4 * i + 2)
        plt.imshow(brain_img / 255.0)
        plt.imshow(brain_heatmap_concat, cmap='jet', alpha=0.5)
        plt.title(f'Grad-CAM Concat: Brain Image - {category}')

        plt.subplot(len(categories), 4, 4 * i + 3)
        plt.imshow(bone_img / 255.0)
        plt.imshow(bone_heatmap_concat, cmap='jet', alpha=0.5)
        plt.title(f'Grad-CAM Concat: Bone Image - {category}')
        
        # Display the Grad-CAM heatmap overlayed on the original image (Product Model)
        plt.subplot(len(categories), 4, 4 * i + 4)
        plt.imshow(brain_img / 255.0)
        plt.imshow(brain_heatmap_product, cmap='jet', alpha=0.5)
        plt.title(f'Grad-CAM Product: Brain Image - {category}')

        plt.subplot(len(categories), 4, 4 * i + 5)
        plt.imshow(bone_img / 255.0)
        plt.imshow(bone_heatmap_product, cmap='jet', alpha=0.5)
        plt.title(f'Grad-CAM Product: Bone Image - {category}')

    plt.tight_layout()
    plt.show()
else:
    print("Error: One or both images could not be loaded. Please check the image paths.")